# Basics of the UI.
We go through the basics of how to use RTNI2, throug the calculation of $\mathbb E \left[UAU^*\right]$ for a random unitary matrix $U$.

## Preparations.

In [1]:
# To use RTNI2, import it, perhaps as rtni.
import rtni2 as rtni

# For symbolic calculations.  
from sympy import symbols

In [2]:
# A ymbolic dimension is set.
n = symbols('n')

## Defining the two matrices $U$ and $A$. 

In [3]:
# Create two matrices, specifying the names and dimensions; both are "originals" for "clones" later. 
# You can pick a nickname, but do not have to. 
u = rtni.matrix(name='u', dims=[[n],[n]], nickname='uppsala')
a = rtni.matrix(name='a', dims=[[n],[n]])
print(u)
print(a)

{'tensor_name': 'u', 'tensor_id': 0, 'tensor_nickname': 'uppsala', 'dims': (n, n), 'dims_mat': ((n,), (n,)), 'transpose': False, 'conjugate': False}

{'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'dims': (n, n), 'dims_mat': ((n,), (n,)), 'transpose': False, 'conjugate': False}



Remark.
- <code>name</code> is set for the family (the original and the clones) and <code>nickname</code> for each clone, including the original. 
- <code>'tensor_id'</code> is assigned automatically within each family. 
- It is not mandatory to set <code>nickname</code> but it sometimes makes things easier because recognizing a matrix (or a tensor) by <code>'tensor_name'</code> and <code>'tensor_id'</code> may not be easy. The default nickname is <code>'{tensor_name}_{tensor_id}'</code>
- <code>dims</code> is set as <code>dims=[[output synbolic dimensions],[input symbolic dimensions]]</code>.

In [4]:
# Clone the matrix u and pick a nick name if you like. 
u_star = u.clone(nickname='ulm')

# the clone is made adjoint. 
u_star.adjoint()

print(u_star)

{'tensor_name': 'u', 'tensor_id': 1, 'tensor_nickname': 'ulm', 'dims': (n, n), 'dims_mat': ((n,), (n,)), 'transpose': True, 'conjugate': True}



Remark. 
- It is important to clone u to make u_star, because these two are "the same matrix". 
- <code>adjoint()</code> is just <code>conjugate()</code> and <code>transpose()</code>; "conjugate" means complex conjugate. So one can use the two instead. Since <code>transpose()</code> switches the input side and the output side internally, so <code>transpose()</code> and <code>adjoint()</code> must applied before connecting them. 
- <code>transpose()</code> and <code>adjoint()</code> are not allowed for tensors. 
- Additionally, a tensor will be created by <code>rtni.tensor(name=, dims=, nickname=)</code>, where <code>dims=[symbolic dimensions]</code>.

## Connecting the matrices. 

In [5]:
# realize the matrix multiplication $uu^*$, for example. 
u.inn(0) * u_star.out(0)

Connected.


Remark.
- <code>inn</code> and <code>out</code> indicate the input and output sides, respectively. The number, counting from 0, is the space id for each side. In this example, each side has only one space. Note that <code>in</code> is a reserved keyword in Python, so we use <code>inn</code> instead. 
- <code>matrix</code> gives a wrapper for tensors. One can directly access the tensors and make the same connection by <code>u(1) * u_star(1)</code>. In tensors, the space ids are assigned from the output space to the input space in the original matrix. Now, the first input space of the matrix u is the second space as a tensor, and the first output space of the adjoint matrix is again the second space. This explains two 1's above; Python counts from 0. 

In [6]:
# already connected space cannot be connected again, giving an error message. 
u.inn(0) * a.out(0)

NodeConnectionError: The first node has already been connected to some node. Disconnect them, first.

Remark.
- An already connected space of tensors or matrices cannot be connected before disconnecting them by <code>~</code> or <code>/</code>.

In [7]:
# Disconnect u.inn(0) from a.out(0).
~u.inn(0) 
# or  
# u.inn(0) / a.out(0)

# and try it again to make a multiplication $UA$. 
u.inn(0) * a.out(0)

Disconnecting from {'tensor_name': 'u', 'tensor_id': 1, 'tensor_nickname': 'ulm', 'space_id': 1, 'dim': n, 'is_dangling_end': False, 'side_original': 'in', 'side_space_id': 0}.
Disconnected.
Connected.


In [8]:
u.inn(0) / a.out(0)
u.inn(0) * a.out(0)

Disconnected.
Connected.


In [9]:
# Let us complete the tensor network diagram by realizing $AU^*$
a.inn(0) * u_star.out(0)

Connected.


## Create a system and place all relevant matrices into it. 

In [10]:
# create the whole system with the relevant matrices. 
tensornetworks = rtni.tensornetworks([a, u, u_star])

tensor a clone 0 has been added.
tensor u clone 0 has been added.
tensor u clone 1 has been added.


Remark.
- You do not have to use all tensors and matrices created or cloned. 
- One can add them separately as below. 

In [11]:
tensornetworks = rtni.tensornetworks()
tensornetworks.add([u_star])
tensornetworks.add([u, a])

tensor u clone 1 has been added.
tensor u clone 0 has been added.
tensor a clone 0 has been added.


In [12]:
# See what our system is. 
tensornetworks.show()

Weight:


1


Edges:
{'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 1, 'dim': n, 'is_dangling_end': False, 'side_original': 'in', 'side_space_id': 0}
<->
{'tensor_name': 'u', 'tensor_id': 1, 'tensor_nickname': 'ulm', 'space_id': 1, 'dim': n, 'is_dangling_end': False, 'side_original': 'in', 'side_space_id': 0}

{'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 0, 'dim': n, 'is_dangling_end': False, 'side_original': 'out', 'side_space_id': 0}
<->
{'tensor_name': 'u', 'tensor_id': 0, 'tensor_nickname': 'uppsala', 'space_id': 1, 'dim': n, 'is_dangling_end': False, 'side_original': 'in', 'side_space_id': 0}




Remark.
- "Weight" will be updated by multiplying Weingarten functions and the weights of loops made through the integration. 
- "Edges" shows the connections.

## Integrating over Haar-distributed unitary matrices. 

In [13]:
# Make a copy so that one can start from here again. This step is not necessary.
import copy
tensornetworks_u = copy.deepcopy(tensornetworks)

In [14]:
# Set "u" to be a unitary matrix and integrate the system over it. 
tensornetworks_u.integrate('u', 'unitary')

# Check the result. 
tensornetworks_u.show()

Integrated. We now have 1 tensor networks.

Weight:


1/n


Edges:
{'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 0, 'dim': n, 'is_dangling_end': False, 'side_original': 'out', 'side_space_id': 0}
<->
{'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 1, 'dim': n, 'is_dangling_end': False, 'side_original': 'in', 'side_space_id': 0}

{'tensor_name': 'dg_u', 'tensor_id': 0, 'tensor_nickname': 'dg_uppsala', 'space_id': 0, 'dim': n, 'is_dangling_end': True, 'side_original': 'out', 'side_space_id': 0, 'tensor_name_origonal': 'u'}
<->
{'tensor_name': 'dg_u', 'tensor_id': 1, 'tensor_nickname': 'dg_ulm', 'space_id': 0, 'dim': n, 'is_dangling_end': True, 'side_original': 'out', 'side_space_id': 0, 'tensor_name_origonal': 'u'}




Remark.
- The "Weight" was updated by the Weingarten function. 
- The first edge represents $\operatorname{Tr}A$. 
- The second edge contains tensors whose names start with "dg_", which stands for "dangling". Since the output side of u and the input side of u_star are "dangling edges", RTNI2 added a formal tensor carrying that information. Hence, this is an edge, linking the following two spaces: after reordering the info, 
    1. <code>{'tensor_name_origonal': 'u', 'tensor_id': 0, 'tensor_nickname': 'uppsala',
     'space_id': 0, 'original_side': 'out', 'side_space_id': 0}</code> 
    2. <code>{'tensor_name_origonal': 'u', 'tensor_id': 1, 'tensor_nickname': 'ueno',
     'space_id': 0, 'original_side': 'out', 'side_space_id': 0}</code>.
    
The former is <code>u</code> and the latter is <code>u_star</code> judging from the id numbers or the nicknames. All things considered, this edge represents the identity map from the (first) input space of <code>u_star</code> to the (first) output space of <code>u</code>.

In [15]:
# Improper settings will result in errors. 
tensornetworks_a = copy.deepcopy(tensornetworks)
tensornetworks_a.integrate('a', 'unitary')

TriviallyZero: The numbers of random tensors and their complex conjugates must match.

Remark.
- The error comes from the fact that the number of copies of a is one. 

## Access each tensor network.
Some may want to use the results for further calculations with their own programs. So, here is how to extract the data. 

In [16]:
# In this example, we have only one tensor network. So, to take it out, going back to tensornetworks_u which was integrated:
tensor_network0 = tensornetworks_u[0]

In [17]:
# Get the weight. By default, the dimension of the output side will be chosen. 
weight = tensor_network0.weight(side='out')
weight

1/n

In [18]:
# Get the removal info
print(tensor_network0._info_removed)

{'u': {'random_tensor_name': 'u', 'random_tensor_type': 'unitary', 'is_complex': True, 'is_group': True, 'dims_mat': ((n,), (n,)), 'dims_tensor': (n, n), 'size': 1}}


In [19]:
# Get the edges. 
for edge in tensor_network0._get_edges():
    display(edge)

[{'tensor_name': 'a',
  'tensor_id': 0,
  'tensor_nickname': 'a_0',
  'space_id': 0,
  'dim': n,
  'is_dangling_end': False,
  'side_original': 'out',
  'side_space_id': 0},
 {'tensor_name': 'a',
  'tensor_id': 0,
  'tensor_nickname': 'a_0',
  'space_id': 1,
  'dim': n,
  'is_dangling_end': False,
  'side_original': 'in',
  'side_space_id': 0}]

[{'tensor_name': 'dg_u',
  'tensor_id': 0,
  'tensor_nickname': 'dg_uppsala',
  'space_id': 0,
  'dim': n,
  'is_dangling_end': True,
  'side_original': 'out',
  'side_space_id': 0,
  'tensor_name_origonal': 'u'},
 {'tensor_name': 'dg_u',
  'tensor_id': 1,
  'tensor_nickname': 'dg_ulm',
  'space_id': 0,
  'dim': n,
  'is_dangling_end': True,
  'side_original': 'out',
  'side_space_id': 0,
  'tensor_name_origonal': 'u'}]

Remark. 
- Each edge is a list of two elements. Each element exlpains which space it corresponds to, in the form of <code>dict</code>. 

## Access each tensor.

In [20]:
# Get the remaining tensors. 
for i in range(len(tensor_network0)):
    print(i)
    display(tensor_network0[i])

0


{'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'dims': (n, n), 'dims_mat': ((n,), (n,)), 'transpose': False, 'conjugate': False}

1


{'tensor_name': 'dg_u'}

In [21]:
# or
for tensor in tensor_network0._get_tensors():
    display(tensor)

{'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'dims': (n, n), 'dims_mat': ((n,), (n,)), 'transpose': False, 'conjugate': False}

{'tensor_name': 'dg_u'}

In [22]:
# Take each.
tensor0 = tensor_network0[0]; print(type(tensor0))
tensor1 = tensor_network0[1]; print(type(tensor1))

<class 'rtni2.Matrix'>
<class 'rtni2._DanglingTensor'>


## Access each space (In PyRTNI2 it is defined by Node).
Note that Tensor and DanglingTensor are differet.

In [23]:
# Work on the usual tensor.
for i in range(len(tensor0)):
    print(i)
    display(tensor0[i])

0


{'this': {'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 0, 'dim': n, 'is_dangling_end': False, 'side_original': 'out', 'side_space_id': 0}, 'connected': {'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 1, 'dim': n, 'is_dangling_end': False, 'side_original': 'in', 'side_space_id': 0}}

1


{'this': {'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 1, 'dim': n, 'is_dangling_end': False, 'side_original': 'in', 'side_space_id': 0}, 'connected': {'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 0, 'dim': n, 'is_dangling_end': False, 'side_original': 'out', 'side_space_id': 0}}

In [24]:
# Or
for node in tensor0.get_nodes():
    display(node)

{'this': {'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 0, 'dim': n, 'is_dangling_end': False, 'side_original': 'out', 'side_space_id': 0}, 'connected': {'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 1, 'dim': n, 'is_dangling_end': False, 'side_original': 'in', 'side_space_id': 0}}

{'this': {'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 1, 'dim': n, 'is_dangling_end': False, 'side_original': 'in', 'side_space_id': 0}, 'connected': {'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 0, 'dim': n, 'is_dangling_end': False, 'side_original': 'out', 'side_space_id': 0}}

In [25]:
# take the first space.
e0 = tensor0[0]; print(e0)

{'this': {'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 0, 'dim': n, 'is_dangling_end': False, 'side_original': 'out', 'side_space_id': 0}, 'connected': {'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 1, 'dim': n, 'is_dangling_end': False, 'side_original': 'in', 'side_space_id': 0}}


In [26]:
# get the one the space is connected to
e0to = e0.get_connected_to(); print(e0to)

{'this': {'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 1, 'dim': n, 'is_dangling_end': False, 'side_original': 'in', 'side_space_id': 0}, 'connected': {'tensor_name': 'a', 'tensor_id': 0, 'tensor_nickname': 'a_0', 'space_id': 0, 'dim': n, 'is_dangling_end': False, 'side_original': 'out', 'side_space_id': 0}}


In [27]:
# Make sure it works. 
e0to == tensor0[1]

True

In [28]:
# Get the tensor back.
tensorx = e0.get_tensor()
tensorx == tensor0

True

In [29]:
# Work on the dangling tensor.
# One DanglingTensor is created for one "family" name.
print(type(tensor1))
print(tensor1)

<class 'rtni2._DanglingTensor'>
{'tensor_name': 'dg_u'}



In [30]:
# Get all spaces.
for i in range(len(tensor1)):
    print(i)
    display(tensor1[i])

0


{'this': {'tensor_name': 'dg_u', 'tensor_id': 1, 'tensor_nickname': 'dg_ulm', 'space_id': 0, 'dim': n, 'is_dangling_end': True, 'side_original': 'out', 'side_space_id': 0, 'tensor_name_origonal': 'u'}, 'connected': {'tensor_name': 'dg_u', 'tensor_id': 0, 'tensor_nickname': 'dg_uppsala', 'space_id': 0, 'dim': n, 'is_dangling_end': True, 'side_original': 'out', 'side_space_id': 0, 'tensor_name_origonal': 'u'}}

1


{'this': {'tensor_name': 'dg_u', 'tensor_id': 0, 'tensor_nickname': 'dg_uppsala', 'space_id': 0, 'dim': n, 'is_dangling_end': True, 'side_original': 'out', 'side_space_id': 0, 'tensor_name_origonal': 'u'}, 'connected': {'tensor_name': 'dg_u', 'tensor_id': 1, 'tensor_nickname': 'dg_ulm', 'space_id': 0, 'dim': n, 'is_dangling_end': True, 'side_original': 'out', 'side_space_id': 0, 'tensor_name_origonal': 'u'}}

In [31]:
# or
for node in tensor1.get_nodes():
    display(node)

{'this': {'tensor_name': 'dg_u', 'tensor_id': 1, 'tensor_nickname': 'dg_ulm', 'space_id': 0, 'dim': n, 'is_dangling_end': True, 'side_original': 'out', 'side_space_id': 0, 'tensor_name_origonal': 'u'}, 'connected': {'tensor_name': 'dg_u', 'tensor_id': 0, 'tensor_nickname': 'dg_uppsala', 'space_id': 0, 'dim': n, 'is_dangling_end': True, 'side_original': 'out', 'side_space_id': 0, 'tensor_name_origonal': 'u'}}

{'this': {'tensor_name': 'dg_u', 'tensor_id': 0, 'tensor_nickname': 'dg_uppsala', 'space_id': 0, 'dim': n, 'is_dangling_end': True, 'side_original': 'out', 'side_space_id': 0, 'tensor_name_origonal': 'u'}, 'connected': {'tensor_name': 'dg_u', 'tensor_id': 1, 'tensor_nickname': 'dg_ulm', 'space_id': 0, 'dim': n, 'is_dangling_end': True, 'side_original': 'out', 'side_space_id': 0, 'tensor_name_origonal': 'u'}}

In [32]:
# take the first space.
f0 = tensor1[0]; print(f0)

{'this': {'tensor_name': 'dg_u', 'tensor_id': 1, 'tensor_nickname': 'dg_ulm', 'space_id': 0, 'dim': n, 'is_dangling_end': True, 'side_original': 'out', 'side_space_id': 0, 'tensor_name_origonal': 'u'}, 'connected': {'tensor_name': 'dg_u', 'tensor_id': 0, 'tensor_nickname': 'dg_uppsala', 'space_id': 0, 'dim': n, 'is_dangling_end': True, 'side_original': 'out', 'side_space_id': 0, 'tensor_name_origonal': 'u'}}


In [33]:
# get the one the space is connected to
f0to = f0.get_connected_to(); print(f0to)

{'this': {'tensor_name': 'dg_u', 'tensor_id': 0, 'tensor_nickname': 'dg_uppsala', 'space_id': 0, 'dim': n, 'is_dangling_end': True, 'side_original': 'out', 'side_space_id': 0, 'tensor_name_origonal': 'u'}, 'connected': {'tensor_name': 'dg_u', 'tensor_id': 1, 'tensor_nickname': 'dg_ulm', 'space_id': 0, 'dim': n, 'is_dangling_end': True, 'side_original': 'out', 'side_space_id': 0, 'tensor_name_origonal': 'u'}}


In [34]:
# Make sure it works. 
f0to == tensor1[1]

True

In [35]:
# Get the dangling tensor back.
tensor_y = f0.get_tensor()
tensor_y == tensor1

True